In [16]:
import pandas as pd
import numpy as np
import os
import tqdm
import warnings

from concurrent.futures import ThreadPoolExecutor

In [17]:
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [11]:
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df

In [18]:
DATA_PATH = '../data/'

In [21]:
train = pd.read_csv(f'{DATA_PATH}/train.csv')
test = pd.read_csv(f'{DATA_PATH}/test.csv')
sample = pd.read_csv(f'{DATA_PATH}/sample_submission.csv')

train_ts = load_time_series(f'{DATA_PATH}/series_train.parquet')
test_ts = load_time_series(f'{DATA_PATH}/series_test.parquet')

df_train = train_ts.drop('id', axis=1)
df_test = test_ts.drop('id', axis=1)

TypeError: 'module' object is not callable